# Checking option greeks and alert

Using webull to get option greeks, then alert by email if greeks exceeding threshold. 

* webull api: https://github.com/tedchou12/webull/blob/775f64632475999bd7ef3d06e19d4c8970b83c47/webull/webull.py#L74
* email alert: https://realpython.com/python-send-email/

You need
* A dummy webull account, suggest not using the real account for trading(security...), signup using email, 1min work.
* A dummy gmail account, you need to turn on less secruity at account page. Using dummy to avoid too many traffic to mess up your regular gmail account.  

### Install webull api
Sometimes, pip install in cmd not working (notebook cannot import), so we directly install from notebook

In [ ]:
import sys

In [ ]:
# directly within Jupyter using the following command in a Jupyter cell
# https://stackoverflow.com/questions/50939461/cant-import-the-installed-package-in-python3-environment-of-jupyter-notebook

# add path
# sys.path.append('C:\\Users\\chuti\\AppData\\Roaming\\Python\\Python37\\Scripts')
# !{sys.executable} -m pip install webull 

### Setup webull api

In [ ]:
from webull import webull 
# from webull import paper_webull

In [ ]:
wb = webull()

In [ ]:
WB_ACCOUNT = input("Type your webull account:")
WB_PSWD = input("Type your password and press enter:")

In [ ]:
# Get temporary code for next step
wb.get_mfa(WB_ACCOUNT) 

In [ ]:
# Use mfa in email from step above
login_info = wb.login(WB_ACCOUNT, WB_PSWD, mfa='954123')

### Setup email for alerts

In [ ]:
import smtplib, ssl

port = 587  # For starttls
smtp_server = "smtp.gmail.com"
SENDER_EMAIL = input("Type your gmail:")
RECEIVER_EMAIL = SENDER_EMAIL
password = input("Type your password and press enter:")

context = ssl.create_default_context()
server = smtplib.SMTP(smtp_server, port)
server.ehlo()  # Can be omitted
server.starttls(context=context)
server.ehlo()  # Can be omitted
server.login(SENDER_EMAIL, password)

# server.sendmail(sender_email, receiver_email, 'hi')

### Params and functions for alert logic

In [ ]:
DELTA = 'delta'
THETA = 'theta'
IV = 'impVol'

THRESHOLD_DELTA_OTM = 0.3
THRESHOLD_DELTA_ITM = 0.7

def delta_alert(cur):
    # put option delta is negative, so use absolute value for checking
    cur = abs(float(cur))
    # otm is too deep
    if cur < THRESHOLD_DELTA_OTM:
        return "!deep otm detected"
    # itm is too deep
    if cur > THRESHOLD_DELTA_ITM:
        return "!deep itm detected"
    
    return ""

class option_greeks:
    def __init__(self, delta, theta, iv):
        self.delta = delta
        self.theta = theta
        self.iv = iv
    
    def to_string(self):
        return f'delta:{self.delta}, theta:{self.theta}, iv:{self.iv}'

### Setup options you want to track

In [ ]:
### Options to track
OPTIONS = [
#    ticker, expireDate, strike, direction, thresholds
    ('TAL', '2021-07-16', '40', 'put'),
    ('TAL', '2021-07-16', '15', 'call'),
]

In [ ]:
# example of response
{'strikePrice': '40', 'call': {'volume': '0', 'latestPriceVol': '0'}, 'put': {'open': '18.51', 'high': '18.89', 'low': '18.51', 'strikePrice': '40', 'preClose': '14.90', 'openInterest': 874, 'volume': '3', 'latestPriceVol': '1', 'delta': '-0.9439', 'vega': '0.0046', 'impVol': '1.8787', 'gamma': '0.0149', 'theta': '-0.0329', 'rho': '-0.0139', 'close': '18.89', 'change': '3.99', 'changeRatio': '0.2678', 'expireDate': '2021-07-16', 'tickerId': 1020908652, 'belongTickerId': 913254311, 'openIntChange': 0, 'activeLevel': 8, 'weekly': 0, 'direction': 'put', 'derivativeStatus': 0, 'currencyId': 247, 'regionId': 6, 'exchangeId': 189, 'symbol': 'TAL210716P00040000', 'unSymbol': 'TAL', 'askList': [{'price': '18.80', 'volume': '143'}], 'bidList': [{'price': '18.40', 'volume': '69'}], 'quoteMultiplier': 100, 'quoteLotSize': 100, 'tradeTime': '2021-07-02T19:48:43.000+0000', 'tradeStamp': 1625255996000}}

In [ ]:
def alert_option_message(options):
    '''
    Job to alert options at each posting
    '''
    alert_messages = []
    for ticker, expireDate, strike, direction in options:
        logs = []
        # option name
        log_option_id = f'{ticker},{expireDate},strike:{strike},{direction}'
        print(f'processing: {log_option_id}')

        # parse option from response
        option_details = wb.get_options_by_strike_and_expire_date(
            stock=ticker,
            expireDate=expireDate,
            strike=strike,
            direction=direction
        )[0]

        greeks = option_greeks(
            delta=option_details[direction][DELTA],
            theta=option_details[direction][THETA],
            iv=option_details[direction][IV]
        )

        # check if alert
        is_alert = False
        causes = []

        # check delta
        delta_alert_msg = delta_alert(greeks.delta)
        if delta_alert_msg:
            is_alert = True
            causes.append(delta_alert_msg)

        if is_alert:
            logs.append(log_option_id)
            logs.append(greeks.to_string())
            logs.append('\n'.join(causes))
            alert_messages.append('\n'.join(logs))
            
    return '\n\n'.join(alert_messages)

In [ ]:
message = alert_option_message(OPTIONS)

In [ ]:
print(message)

In [ ]:
def alert_job(server, options):
    alert_message = alert_option_message(options)
    
    if alert_message:
        message = f"""\
Subject: Option alerts!

{alert_message}"""
        server.sendmail(SENDER_EMAIL, RECEIVER_EMAIL, message)

In [ ]:
alert_job(server, OPTIONS)

### Sechdule the job
https://stackoverflow.com/questions/22715086/scheduling-python-script-to-run-every-hour-accurately

In [ ]:
# install package if not exist
# !{sys.executable} -m pip install apscheduler

In [ ]:
from apscheduler.schedulers.blocking import BlockingScheduler

sched = BlockingScheduler()

@sched.scheduled_job('interval', seconds=10)
def timed_job():
    print('This job is run every 10 seconds.')
    alert_job(server, OPTIONS)
    
sched.start()